In [1]:
import pydicom
import shutil
import glob
import pandas as pd
import os
import numpy as np

In [2]:
BASE_DIR = r"/media/df3/Fetal/Data-All/"
BASE_DIR = r"/media/df3/Fetal/Fetal-Elka/Elka1/bunch/"


In [3]:
ORIENTATION_DIR = {"cor" : "cor",
                   "tra" : "tra",
                   "ax"  : "tra",
                   "sag" : "sag"}

    
def get_geomtric_data(patient_dir):
    df = pd.DataFrame()
    for elem in glob.glob(os.path.join(patient_dir, "*")):
        if not os.path.isdir(elem):
            continue
        elem = elem.split("/")[-1]
        elem_split = elem.split("_")
        if len(elem_split) == 0:
            print ("PASSED")
            continue
        elem_index = int(elem_split[0].split("Se")[1])

        df.loc[elem_index, "Name"] = elem.split("_", 1)[1]
        df.loc[elem_index, "Sequence"] = int(elem_index)
        df.loc[elem_index, "Orientation"] = ORIENTATION_DIR.get(elem_split[1].lower(), None)

        dicomfiles = glob.glob(os.path.join(patient_dir, elem, "*.dcm"))
        if len(dicomfiles) == 0:
            continue
        dicomfile = dicomfiles[0]
        dicomobj = pydicom.read_file(dicomfile)

        df.loc[elem_index, "TE"] = dicomobj.get("EchoTime")
        df.loc[elem_index, "TR"] = dicomobj.get("RepetitionTime")
        df.loc[elem_index, "AcquisitionMatrix"] = str(dicomobj.get("AcquisitionMatrix"))
        df.loc[elem_index, "Rows"] = dicomobj.get("Rows")
        df.loc[elem_index, "Columns"] = dicomobj.get("Columns")
        acq_dim = np.array(dicomobj.get("AcquisitionMatrix"))
        acq_dim = acq_dim[acq_dim != 0]
        df.loc[elem_index, "IsInterpolated"] = dicomobj.Rows * dicomobj.Columns != np.prod(acq_dim)
        df.loc[elem_index, "nSlices"] = len(dicomfiles)
        if hasattr(dicomobj, "PixelSpacing"):
            df.loc[elem_index, "PixelSpacing"] = str([float(a) for a in dicomobj.PixelSpacing])
            df.loc[elem_index, "FOV"] = str([int(df.loc[elem_index,"Rows"] * dicomobj.PixelSpacing[0]), 
                                             int(df.loc[elem_index,"Columns"] * dicomobj.PixelSpacing[0])])
        df.loc[elem_index, "SliceThickness"] = dicomobj.get("SliceThickness")
        if hasattr(dicomobj, "SpacingBetweenSlices"):
            df.loc[elem_index, "SpacingBetweenSlices"] = dicomobj.SpacingBetweenSlices 
    df = df.sort_index()
    return df
    

In [4]:
for patient_dir in glob.iglob(os.path.join(BASE_DIR,"*")):
    if not os.path.isdir(patient_dir) or 'newarrivals' in patient_dir:
        continue
    print (patient_dir)
    df = get_geomtric_data(patient_dir)
    df.to_excel(os.path.join(patient_dir, "series_data.xlsx"))

/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_32
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_49
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_57
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_94
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_14
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_45
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_101
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_71
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_10
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_64
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_80
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_3
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_38
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_23
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_89
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_81
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_92-93
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_99
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_21
/media/df3/Fetal/Fetal-Elka/Elka1/bunch/FETAL_82
/media/df3/Fetal/

In [90]:
get_geomtric_data(os.path.join(BASE_DIR,"SHAHAR_KEIDAR_SHIRAN_20190217_210846323"))

""


In [78]:
df = pd.DataFrame()
i = 0;
for patient_dir in glob.iglob(os.path.join(BASE_DIR,"*")):
    if not os.path.isdir(patient_dir):
        continue
    flag = False
    for filepath in glob.iglob(os.path.join(patient_dir,"*","*.dcm")):
        if flag:
            break
        if filepath.endswith(".dcm"):
            dicomobj = pydicom.read_file(filepath)
            i = i + 1
            df.loc[i, "PatientID"] = dicomobj.PatientID
            df.loc[i, "Modality"] = "MR"
            datestr = dicomobj.StudyDate
            datestr = datestr[6:] + "-" + datestr[4:6] + "-" + datestr[:4]
            df.loc[i, "StudyDateMin"] = datestr
            df.loc[i, "StudyDateMax"] = datestr
            flag = True
    print (patient_dir)
                

/media/df3/Fetal/Data-All/GHANAYIM_SAHERA_20190217_113603840
/media/df3/Fetal/Data-All/OZ_MORAN_SHOSHANA_20190217_150720598
/media/df3/Fetal/Data-All/ABU_EL_OLA_ANWAR_20190217_151225984
/media/df3/Fetal/Data-All/COHEN_AYELET_20190217_152909288
/media/df3/Fetal/Data-All/RACHVALSKY_SHEMESH_AVIA_20190217_154458703
/media/df3/Fetal/Data-All/HADAD_TSOFIT_KEREN_20190217_165300492
/media/df3/Fetal/Data-All/YURAN_LIPSKI_NOFAR_20190217_165528543
/media/df3/Fetal/Data-All/PRIMO_GERSHON_DANA_20190217_165746998
/media/df3/Fetal/Data-All/DABAH_SHIRA_20190217_171326690
/media/df3/Fetal/Data-All/SIBONI_LIOR_20190217_171821057
/media/df3/Fetal/Data-All/ALONI_ILANIT_20190217_172930266
/media/df3/Fetal/Data-All/LAX_ADI_20190217_173602124
/media/df3/Fetal/Data-All/GAL_KALANIT_20190217_173706842
/media/df3/Fetal/Data-All/TEWELDE_MERHAWIT_20190217_174331511
/media/df3/Fetal/Data-All/BENAYCH_YAEL_20190217_174443043
/media/df3/Fetal/Data-All/ALMAN_YARA_20190217_174517302
/media/df3/Fetal/Data-All/YAKIRI_SHIR

/media/df3/Fetal/Data-All/RISKIN_NATALIA_20190217_191200889
/media/df3/Fetal/Data-All/LEVINE_HAGIT_20190217_191212536
/media/df3/Fetal/Data-All/NADAV_BATYA_20190217_192219896
/media/df3/Fetal/Data-All/SOLODUCHO_KASHMIR_ILANA_20190217_193815561
/media/df3/Fetal/Data-All/BEN_DOR_HAMUTAL_20190217_194235842
/media/df3/Fetal/Data-All/KOSTIUTCHENKOV_MARGARITA_20190217_194636371
/media/df3/Fetal/Data-All/BUSKILA_SIGALIT_20190217_195641276
/media/df3/Fetal/Data-All/ELMALIACH_CARMIT_20190217_195751836
/media/df3/Fetal/Data-All/BACMAN_DANA_EVELYNE_20190217_195759065
/media/df3/Fetal/Data-All/GECKT_MORAN_20190217_200023870
/media/df3/Fetal/Data-All/HAUSMAN_SHIRA_20190217_200447595
/media/df3/Fetal/Data-All/KADAN_MALAK_20190217_201354605
/media/df3/Fetal/Data-All/SIMAN_TOV_MENASHE_YIFAT_20190217_201519921
/media/df3/Fetal/Data-All/VISHLIZKI_YAEL_20190217_201604866
/media/df3/Fetal/Data-All/SAIAS_KEREN_20190217_202256916
/media/df3/Fetal/Data-All/AVITAN_RAVIT_20190217_202345276
/media/df3/Fetal/Dat

/media/df3/Fetal/Data-All/ASHKENAZI_DAFNA_20190217_150203150
/media/df3/Fetal/Data-All/LESTREL_DANIELA_AMALIA_20190217_113455449
/media/df3/Fetal/Data-All/BERGER_NECHAMA_20190217_150913389
/media/df3/Fetal/Data-All/SHTILMAN_RACHEL_MANOR_20190217_151623331
/media/df3/Fetal/Data-All/ABU_FARES_NARMIN_20190217_153044887
/media/df3/Fetal/Data-All/KEDMI_ADI_20190217_103542653
/media/df3/Fetal/Data-All/RUDERMAN_ARDEN_SUZY_20190217_153643725
/media/df3/Fetal/Data-All/DOVDOVANI_NEHAMA_20190217_152927534
/media/df3/Fetal/Data-All/VAISBURD_KACHKO_SINAYA_20190217_114141738
/media/df3/Fetal/Data-All/NISIMOV_RIKI_20190217_103803187
/media/df3/Fetal/Data-All/LUZIA_WEINBERG_CHEN_20190217_162456729
/media/df3/Fetal/Data-All/LUZIA_WEINBERG_CHEN_20190217_162503340
/media/df3/Fetal/Data-All/LIVMAN_HALILI_MICHAL_20190217_162640553
/media/df3/Fetal/Data-All/HUGIRAT_AMIRA_20190217_163233658
/media/df3/Fetal/Data-All/PASHAREL_MORAN_20190217_163351968
/media/df3/Fetal/Data-All/WEINSTEIN_SISI_20190217_170147909

/media/df3/Fetal/Data-All/PERETZ_DAVID_SHANI_20190219_133706670
/media/df3/Fetal/Data-All/POLAD_ELENA_20190219_141724897
/media/df3/Fetal/Data-All/ROZNAK_SHIRAN_20190219_141019941
/media/df3/Fetal/Data-All/SABISKI_JOANA_SONIA_20190219_114915174
/media/df3/Fetal/Data-All/SMADIA_YAFA_20190219_141514288
/media/df3/Fetal/Data-All/TOIBIN_GAL_TAL_20190219_133803822
/media/df3/Fetal/Data-All/ZAHRAWI_AMANI_20190219_133528126
/media/df3/Fetal/Data-All/VROBEL_PEERY_MORAN_20190217_151914839
/media/df3/Fetal/Data-All/SHARABI_SHIRLY_20190217_150714133
/media/df3/Fetal/Data-All/GOTLIB_BAR_20190217_150823353
/media/df3/Fetal/Data-All/HAIMOV_BATEL_20190217_152740851
/media/df3/Fetal/Data-All/ESHED_NOAM_20190217_114319872
/media/df3/Fetal/Data-All/RABENKOV_BARBARA_20190217_152104638
/media/df3/Fetal/Data-All/BAR_ZIV_GIL_20190217_113538496
/media/df3/Fetal/Data-All/BEN_BASAT_SHANY_20190217_150327406
/media/df3/Fetal/Data-All/DAMTI_NETA_20190217_154139727
/media/df3/Fetal/Data-All/SULIMAN_ODIYA_20190217_

/media/df3/Fetal/Data-All/YHOODAYOFF_TALIA_20190217_202522083
/media/df3/Fetal/Data-All/MANIHAT_TAGRID_20190217_203813288
/media/df3/Fetal/Data-All/DAHAN_HODAIA_HANA_20190217_203818266
/media/df3/Fetal/Data-All/COHEN_HILA_20190217_204349261
/media/df3/Fetal/Data-All/BER_DVORAH_20190217_205000538
/media/df3/Fetal/Data-All/DANIN_PASTER_LIOR_20190217_210038264
/media/df3/Fetal/Data-All/RAZ_MICHAL_20190217_210108817
/media/df3/Fetal/Data-All/DARY_LIAT_20190217_210346202
/media/df3/Fetal/Data-All/SOLOMON_KEREN_20190217_210401172
/media/df3/Fetal/Data-All/ZIV_LEV_ADI_20190217_210823021
/media/df3/Fetal/Data-All/HAZAN_ADI_20190217_210842223
/media/df3/Fetal/Data-All/KESLER_KEREN_20190219_133924976
/media/df3/Fetal/Data-All/LEVCHENKO_KRISTINA_20190219_115411942
/media/df3/Fetal/Data-All/ROZENFELD_RAISSA_20190217_150408251
/media/df3/Fetal/Data-All/BARBIRO_KEREN_20190217_163919420
/media/df3/Fetal/Data-All/EYLON_MERAV_20190217_154223441
/media/df3/Fetal/Data-All/KAWA_MILA_AUDREY_20190217_113958

...

/media/df3/Fetal/Data-All/ZUCHMAN_ORLY_20190217_185228542
/media/df3/Fetal/Data-All/WEINGARDEN_MERAV_20190217_203247696
/media/df3/Fetal/Data-All/HALEVY_DAFNA_20190217_195901894
/media/df3/Fetal/Data-All/TAL_CHEN_SIMA_20190217_181729892
/media/df3/Fetal/Data-All/ASHUR_JASMINE_AVITAL_20190217_172147384
/media/df3/Fetal/Data-All/MIZRACHI_ELIANA_ELISHEVA_20190217_154415174
/media/df3/Fetal/Data-All/AIALA_ENKOIE_20190217_151326196
/media/df3/Fetal/Data-All/TOKAREV_MARINA_20190217_175407014
/media/df3/Fetal/Data-All/ENOCH_KOTLER_AVIV_20190217_200842741
/media/df3/Fetal/Data-All/SIMAAN_MARIA_20190217_172950294
/media/df3/Fetal/Data-All/ZUR_RONI_20190217_203237194
/media/df3/Fetal/Data-All/KEDMI_ADI_20190217_113805310
/media/df3/Fetal/Data-All/COHEN_IDIT_20190217_174631523
/media/df3/Fetal/Data-All/ODEH_SHIREEN_20190217_180115083
/media/df3/Fetal/Data-All/KAHALANI_HADAR_20190217_201626628
/media/df3/Fetal/Data-All/NIMNI_MAGI_MARGALIT_20190217_210203253
/media/df3/Fetal/Data-All/LEVKOVITCH_FOR

/media/df3/Fetal/Data-All/KOENIG_ZOBEL_ANAT_20190217_153523016
/media/df3/Fetal/Data-All/MISHKAL_REVITAL_20190217_204412793
/media/df3/Fetal/Data-All/GECKT_MORAN_20190217_200019321
/media/df3/Fetal/Data-All/ASHWAL_SHLOMIT_20190217_165030798
/media/df3/Fetal/Data-All/PERETZ_SIVAN_20190217_164916217
/media/df3/Fetal/Data-All/OHYOON_GAL_20190217_194406454
/media/df3/Fetal/Data-All/ZOHAR_ILANIT_TAMAR_20190217_173159720
/media/df3/Fetal/Data-All/GAL_EINAT_20190217_202201707
/media/df3/Fetal/Data-All/AHARON_BRITTANY_20190217_201825812
/media/df3/Fetal/Data-All/GAL_EINAT_20190217_202201641
/media/df3/Fetal/Data-All/COHEN_EINAV_20190217_191407529
/media/df3/Fetal/Data-All/MASALA_ZIV_20190217_171820302
/media/df3/Fetal/Data-All/HENIG_OSHRAT_20190217_201529806
/media/df3/Fetal/Data-All/PENN_NOA_20190217_164214787
/media/df3/Fetal/Data-All/BITON_TSLIL_20190217_210916525
/media/df3/Fetal/Data-All/RAJCHER_SABINA_JUDITH_20190217_210946584
/media/df3/Fetal/Data-All/GERSHON_RUTH_REUT_20190219_14100537

/media/df3/Fetal/Data-All/HEN_MEITAL_20190217_202607252
/media/df3/Fetal/Data-All/BITON_SHANI_20190217_204841607
/media/df3/Fetal/Data-All/BENGIO_SHARON_20190217_203618049
/media/df3/Fetal/Data-All/ISREL_HAGAMA_HAGAR_20190217_172221755
/media/df3/Fetal/Data-All/KEDEM_YAFIT_20190217_172414623
/media/df3/Fetal/Data-All/KLEIN_GOLDENBERG_ADI_20190217_191054722
/media/df3/Fetal/Data-All/MOSHEL_RAVENSARY_DVORA_LEA_20190217_203258165
/media/df3/Fetal/Data-All/OMESI_MIZRACHI_CALANIT_20190217_201531126
/media/df3/Fetal/Data-All/KADAR_LALIV__LILACH_20190217_204806173
/media/df3/Fetal/Data-All/AMIR_KOREN_LIRON_20190219_114335816
/media/df3/Fetal/Data-All/KAPLAN_RAVIV_SHANI_20190217_165514175
/media/df3/Fetal/Data-All/YEHEZKEL_SOL_KEREN_OR_20190217_200745091
/media/df3/Fetal/Data-All/PANIGEL_MAAYAN_20190217_172422405
/media/df3/Fetal/Data-All/COHEN_JULIE_GEULA_20190217_202551841
/media/df3/Fetal/Data-All/SHIK_NOA_20190217_162933672
/media/df3/Fetal/Data-All/BOUSKILA_YAEL_OLIVIA_20190217_175246416


/media/df3/Fetal/Data-All/MARUM_ANAT_20190217_211050279
/media/df3/Fetal/Data-All/FRIEDENSOHN_ZUCK_LIMOR_20190219_133913310
/media/df3/Fetal/Data-All/HATIV_MANHAL_20190219_134302967
/media/df3/Fetal/Data-All/ITZHAKOV_MANDANA_20190219_141919705
/media/df3/Fetal/Data-All/NAGAYEV_IRIT_20190219_142014593
/media/df3/Fetal/Data-All/QUITT_NURIT_20190219_115856042
/media/df3/Fetal/Data-All/RAHAMIM_OSNAT_20190219_133534879
/media/df3/Fetal/Data-All/TSITLOVSKI_NELYA_NESYA_20190219_133538060
/media/df3/Fetal/Data-All/ATAR_ZEISEL_LITAL_20190217_152310369
/media/df3/Fetal/Data-All/BENUN_OSNAT_20190217_171055454
/media/df3/Fetal/Data-All/PERETZ_ATARA_20190217_181858605
/media/df3/Fetal/Data-All/TZALMON_PERY_AMANDA_RACHEL_20190217_163413432
/media/df3/Fetal/Data-All/COREN_LIORA_20190217_205911611
/media/df3/Fetal/Data-All/ZALCBERG_MORAN_20190217_181058117
/media/df3/Fetal/Data-All/NASSRALLAH_HADDAD_RAWAN_20190217_154909805
/media/df3/Fetal/Data-All/GRISARO_KREISERMAN_ORIT_20190217_204425403
/media/df

/media/df3/Fetal/Data-All/PONKRATOV_EVGENIA_20190217_182635836
/media/df3/Fetal/Data-All/LEVI_SHIR_SARA_20190217_161800302
/media/df3/Fetal/Data-All/WISHINSKI_NEOMI_20190217_182340980
/media/df3/Fetal/Data-All/EROS_MICHAL_20190217_204530653
/media/df3/Fetal/Data-All/ELAD_INBAL_ANNABEL_20190217_203808221
/media/df3/Fetal/Data-All/NASHEF_HUDA_20190217_182705892
/media/df3/Fetal/Data-All/YANAY_GAL_20190217_191537889
/media/df3/Fetal/Data-All/YARDEN_TAL_20190217_173354859
/media/df3/Fetal/Data-All/KHENKEL_VALERIIA_20190217_175659004
/media/df3/Fetal/Data-All/HIRSCHHORN_LIAT_BLUMA_20190217_152403056
/media/df3/Fetal/Data-All/COHEN_GAL_20190217_203738899
/media/df3/Fetal/Data-All/SOLODUCHO_KASHMIR_ILANA_20190217_154528721
/media/df3/Fetal/Data-All/MAOZ_SHARON_20190217_153033595
/media/df3/Fetal/Data-All/ZERAPHA_SHAUL_RUTH_SARA_20190217_180106119
/media/df3/Fetal/Data-All/WEISS_LORIN_20190217_202250970
/media/df3/Fetal/Data-All/KAMINSKY_MARINA_20190217_185309151
/media/df3/Fetal/Data-All/ABOU

In [79]:
df.to_excel("/media/df3/Fetal/n_gaddi1.xlsx")